In [1]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Numeric, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy.schema import CreateTable
from sqlalchemy.sql.expression import insert
from faker import Faker
import os
import random
import datetime

In [2]:
RANDOM_SEED = 42
SAVE_DIR = "../flyway/migrations"

NB_CUSTOMERS = 500
NB_PRODUCTS = 50
NB_DEVICES = 500
NB_TRANSACTIONS = 2000
START_DATE = datetime.date(2018, 1, 1)
END_DATE = datetime.date(2023, 6, 30)

In [3]:
# Create a declarative base class
Base = declarative_base()

# Create a Faker instance
Faker.seed(RANDOM_SEED)
fake = Faker()

# Set random seed
random.seed(RANDOM_SEED)

## Define schemas

In [4]:
class Customer(Base):
    __tablename__ = "customers"
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    email = Column(String(100), nullable=False)
    phone = Column(String(25), nullable=False)
    address = Column(String(250), nullable=False)
    city = Column(String(100), nullable=False)
    country = Column(String(100), nullable=False)

    def to_dict(self):
        user_dict = vars(self).copy()
        user_dict.pop("_sa_instance_state")
        return user_dict

In [5]:
class Product(Base):
    __tablename__ = "products"
    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)
    description = Column(String(200), nullable=False)
    category = Column(String(50), nullable=False)
    price = Column(Numeric(10, 2), nullable=False)

    def to_dict(self):
        user_dict = vars(self).copy()
        user_dict.pop("_sa_instance_state")
        return user_dict

In [6]:
class Device(Base):
    __tablename__ = "devices"
    id = Column(Integer, primary_key=True)
    platform = Column(String(250), nullable=False)
    ipv4 = Column(String(50), nullable=False)
    macaddress = Column(String(50), nullable=False)

    def to_dict(self):
        user_dict = vars(self).copy()
        user_dict.pop("_sa_instance_state")
        return user_dict

In [7]:
class Transaction(Base):
    __tablename__ = "transactions"
    id = Column(Integer, primary_key=True)
    date_time = Column(DateTime, nullable=False)
    customer_id = Column(Integer, ForeignKey("customers.id"))
    product_id = Column(Integer, ForeignKey("products.id"))
    quantity = Column(Integer, nullable=False)
    device_id = Column(Integer, ForeignKey("devices.id"))
    payment_method = Column(String(50), nullable=False)

    def to_dict(self):
        user_dict = vars(self).copy()
        user_dict.pop("_sa_instance_state")
        return user_dict

## Instance generators

In [8]:
def generate_customer(id: int):
    customer = Customer(
        id=id,
        name=fake.name(),
        email=fake.email(),
        phone=fake.phone_number(),
        address=fake.street_address(),
        city=fake.city(),
        country=fake.country()
    )
    return customer

In [9]:
def generate_product(id: int):
    product = Product(
        id=id,
        name=fake.word(),
        description=fake.sentence(nb_words=10, ext_word_list=["best", "worst", "the", "this", "product", "is", "favorite", "ever"]),
        category=fake.random_element(elements=("Electronics", "Fashion", "Books", "Garden", "Games", "Sports", "Food")),
        price=fake.pydecimal(left_digits=3, right_digits=2, positive=True)
    )
    return product

In [10]:
def generate_device(id: int):
    device = Device(
        id=id,
        platform=fake.user_agent(),
        ipv4=fake.ipv4(),
        macaddress=fake.mac_address()
    )
    return device

In [11]:
def generate_transaction(id: int, customers: list[Customer], products: list[Product], devices:list[Device]):
    tr = Transaction(
        id=id,
        date_time=fake.date_between(start_date=START_DATE, end_date=END_DATE),
        customer_id=random.choice(customers).id,
        product_id=random.choice(products).id,
        quantity=fake.random_int(min=1, max=20),
        device_id=random.choice(devices).id,
        payment_method=fake.random_element(elements=("Credit Card", "EFT", "Bitcoin", "Reward Points"))
    )
    return tr

## Create data

In [12]:
%%time
customers = [generate_customer(i) for i in range(NB_CUSTOMERS)]
products = [generate_product(i) for i in range(NB_PRODUCTS)]
devices = [generate_device(i) for i in range(NB_DEVICES)]
transactions = [generate_transaction(i, customers, products, devices) for i in range(NB_TRANSACTIONS)]

CPU times: user 200 ms, sys: 2.61 ms, total: 202 ms
Wall time: 204 ms


## Save data to SQL create scripts

In [13]:
# Create a function to save the sql code to a file
def save_sql_file(sql, filename, save_dir="./sql"):
    os.makedirs(save_dir, exist_ok=True)
    with open(os.path.join(save_dir, filename), "w") as f:
        f.write(sql)

In [14]:
# Create an engine
engine = create_engine("postgresql:///:memory:")

In [15]:
# Create dictionary that contains all data
data_dict = {
    "customers": {
        "class": Customer,
        "records": customers
    },
    "products": {
        "class": Product,
        "records": products
    },
    "devices": {
        "class": Device,
        "records": devices
    },
    "transactions": {
        "class": Transaction,
        "records": transactions
    }
}

In [16]:
# Create the sql scripts
script_num = 1
for table, details in data_dict.items():
    cls = details["class"]
    table_name = cls.__tablename__
    records = details["records"]

    print(f"Table: {table_name}")
    print("\tGenerating create script...")
    create_table_script = str(CreateTable(cls.__table__).compile(engine))
    save_sql_file(create_table_script, f"V1_{script_num}__create_{table_name}_table.sql", SAVE_DIR)
    script_num += 1

    print("\tGenerating insert script...")
    insert_stmt = insert(cls.__table__)\
        .values([r.to_dict() for r in records])\
        .compile(compile_kwargs={"literal_binds": True})
    insert_stmt = str(insert_stmt)
    save_sql_file(insert_stmt, f"V1_{script_num}__insert_{table_name}_records.sql", SAVE_DIR)
    script_num += 1


Table: customers
	Generating create script...
	Generating insert script...
Table: products
	Generating create script...
	Generating insert script...
Table: devices
	Generating create script...
	Generating insert script...
Table: transactions
	Generating create script...
	Generating insert script...
